<a href="https://colab.research.google.com/github/andrePankraz/qa_service/blob/main/notebooks/Neustarthilfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neustarthilfe 2022 FAQ-Suche
Install all necessary packages.

In [2]:
!pip install --quiet aleph-alpha-client openai stanza tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.5/802.5 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 98.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Import text data:
*   Fetch URL web page with FAQ text and parse HTML
*   Extract FAQ text data into data struct (list of lists)
*   Export as JSON lines file 'faq.jsonl' and as text file 'faq.txt'

In [3]:
from bs4 import BeautifulSoup, NavigableString
from dataclasses import dataclass
import json
import re
import requests

@dataclass
class Faq:
    id: int
    group_id: int
    group_title: str
    question_id: int
    question: str
    answer: str

# Convert HTML element into raw inner text:
# Preserve some structure like paragraphs, lists etc.
def process_element(element, indent=''):
    if isinstance(element, NavigableString):
        return element # preserve <span>&nbsp;</span>

    if element.name == 'p':
        content = ''.join(process_element(child, indent) for child in element.children)
        return f'\n{indent}{content.strip()}\n'

    if element.name == 'br':
        return f'\n{indent}'

    if element.name in ['ul', 'ol']:
        items = ['\n']
        for item in element.find_all('li', recursive=False):
            item_content = ''.join(process_element(child, indent + '    ') for child in item.children)
            items.append(f'{indent}- {item_content.strip()}')
        return '\n'.join(items)

    if element.name == 'sup':
        return ''

    if element.name == 'tr':
        items = []
        for item in element.find_all('td', recursive=False):
            item_content = ''.join(process_element(child, '') for child in item.children)
            items.append(item_content)
        return '\n' + ' | '.join(items)

    # Process other elements and concatenate their content
    content = []
    for child in element.children:
        content.append(process_element(child, indent))
    return ''.join(content)

# Convert FAQ texts from source HTML into a FAQ data struct (list of lists)
def extract_elterngeld_digital_faq() -> list[Faq]:
    # Fetch HTML with FAQ texts:
    url = 'https://www.ueberbrueckungshilfe-unternehmen.de/DE/FAQ/Nsh-22/neustarthilfe-2022.html'
    response = requests.get(url)
    content = response.content.decode('utf-8')

    # Remove potential carriage returns
    content = content.replace('\r', '')
    # Replace newline characters with a space (we have no <pre>)
    content = content.replace('\n', ' ')
    # Replace multiple spaces with a single space
    content = re.sub(r'\s+', ' ', content)

    # Parse HTML
    soup = BeautifulSoup(content, 'html.parser')

    # Target data structure
    faq = [Faq('ID', 'Thema_ID', 'Thema', 'Frage_ID', 'Frage', 'Antwort')]

    # Get relevant root element for FAQ texts
    main_div_element = soup.find('div', class_='accordion__content')

    # Extract FAQ texts into data structure
    id = 0
    group_id = 0
    question_id = 0

    for child in main_div_element.children:
      if child.name == 'h2' and child.get('class') == ['accordion__headline']:
        group_id += 1
        question_id = 0
        group_title = child.text.strip().split(' ', 1)[1]
      elif child.name == 'div' and child.get('class') == ['accordion__element']:
        id += 1
        question_id += 1
        question = child.find('h3', class_='accordion__title').text.strip().split(' ', 1)[1]
        answer_element = child.find('div', class_='accordion__panel')
        answer = process_element(answer_element)
        # Replace multiple "empty lines" (lines with just spaces and \n) with a single newline character
        # and replace "trailing spaces followed by \n" with just "\n"
        answer = re.sub(r'([ ]*\n)+', '\n', answer).strip()
        faq.append(Faq(id, group_id, group_title, question_id, question, answer))

    return faq

# Export FAQ data struct as JSON lines file
def write_faq(file, faq: list[Faq]):
    with open(file, 'w', encoding='utf-8') as f:
      for entry in faq:
        f.write(json.dumps(list(entry.__dict__.values()), ensure_ascii=False) + '\n')

# Export FAQ data struct as raw text file (for debugging)
def write_faq_text(faq: list[Faq]):
    with open('faq.txt', 'w', encoding='utf-8') as f:
      for entry in faq[1:]:
        f.write(f"{entry.group_id}.{entry.question_id} {entry.question}\n{entry.answer}\n")
        f.write('\n')

# Call this functions
faq = extract_elterngeld_digital_faq()
write_faq('faq.json', faq)
write_faq_text(faq)

# Print migrated data for debugging
if False:
    for item in faq:
      print(f"{item[4]}\n{item[5]}\n")

Import JSON lines file 'faq.jsonl' and convert into generic format for document question answering.

In [4]:
def load_faq(file: str) -> list[Faq]:
    faq = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            faq.append(Faq(*json.loads(line.strip())))
    return faq

faq: list[Faq] = load_faq('faq.json')

# Open Source Models (On-Prem)
## Retrieval and Embeddings
Bi-Encoder models (or embedding models) create high dimensional vectors from question and paragraphs (embeddings). Similar embeddings show semantic relationships and enable retrieval of facts from large documents.

Load embedding model.

In [5]:
import torch
from sentence_transformers import SentenceTransformer

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Model Size is around 1.3 GB, Multilingual,
# Max sequence length is 512 -> Embedding is 1024 dimensional
embedding_model_id = 'aari1995/German_Semantic_STS_V2'

embedding_model = SentenceTransformer(embedding_model_id, device=device)
embedding_max_seq_length = embedding_model.max_seq_length

embedding_model.device, embedding_model, embedding_max_seq_length, embedding_model.encode('Test', convert_to_tensor=True).size()

(device(type='cpu'),
 SentenceTransformer(
   (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
   (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
 ),
 512,
 torch.Size([1024]))

Load NLP models for sentence splitting.

In [6]:
import stanza

# configure stanza for sentence splitting in multiple languages
nlp = stanza.MultilingualPipeline(
    lang_id_config={'langid_clean_text': True},
    lang_configs={'de': {'processors': 'tokenize,mwt', 'verbose': False}, 'en': {'processors': 'tokenize', 'verbose': False}})
test = nlp('Initialisiere Deutsche Modelle. Das ist ein Test').sentences

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: langid
INFO:stanza:Done loading processors!


Split documents into paragraphs (chunks / windows) for embedding:
*   Embedding models have a max token size, use it for splitting
*   Split at sentence ends, not in middle of sentences
*   Overlap chunks if possible

In [7]:
@dataclass
class Paragraph:
    id: int
    sentence: int
    tokens: int
    external_id: str
    title: str
    text: str

def split_into_paragraphs(documents, max_sentences, overlap_sentences, tokenizer, max_tokens, overlap_tokens) -> list[Paragraph]:
    paragraphs = []  # resulting list

    for document in documents:
        id, external_id, title, text = document

        # pre-calculate token number for document title (part of embedding paragraph)
        title_tokens = len(tokenizer.encode(title))

        # split document text into sentences
        nlp_sentences = [s.text for s in nlp(text).sentences]
        # pre-calculate token numbers for each document sentence
        sentence_tokens = [len(tokenizer.encode(nlp_sentence)) for nlp_sentence in nlp_sentences]

        sentence_index = -1
        paragraph = ''
        paragraph_sentences = 0
        paragraph_tokens = 0

        index = 0
        while index < len(sentence_tokens):
            tokens = sentence_tokens[index]

            if sentence_index == -1:
                # start new paragraph
                sentence_index = index
                paragraph = nlp_sentences[index]
                paragraph_sentences = 1
                paragraph_tokens = title_tokens + 1 + tokens
                index += 1
                continue

            if (max_sentences <= 0 or paragraph_sentences < max_sentences) and (max_tokens <= 0 or paragraph_tokens + tokens <= max_tokens):
                # continue paragraph
                paragraph += ' ' + nlp_sentences[index]
                paragraph_sentences += 1
                paragraph_tokens += 1 + tokens
                index += 1
                continue

            # finish paragraph
            paragraphs.append(Paragraph(id, sentence_index, paragraph_tokens, external_id, title, paragraph))

            # overlap paragraphs with sentence or token window - whatever boundary triggered first
            if max_sentences > 0 and paragraph_sentences == max_sentences and overlap_sentences <= max_sentences / 2:
              index -= overlap_sentences
            if max_tokens > 0 and paragraph_tokens + tokens > max_tokens and overlap_tokens > 0 and overlap_tokens <= max_tokens / 2:
                overlap_tokens_sum = 0
                while index > sentence_index + 1:
                    overlap_tokens_sum += sentence_tokens[index - 1]
                    if overlap_tokens_sum > overlap_tokens:
                        break
                    index -= 1

            # trigger new paragraph
            sentence_index = -1
        else:
            if sentence_index != -1:
                # add final paragraph
                paragraphs.append(Paragraph(id, sentence_index, paragraph_tokens, external_id, title, paragraph))
    return paragraphs


# Convert to simpler datamodel: [ID, Title, Text]
documents = [(f.id, f"{f.group_id}.{f.question_id}", f.question, f.answer) for f in faq[1:]]

max_sentences = 6
overlap_sentences = 1
max_tokens = embedding_max_seq_length
overlap_tokens = max_tokens / 6

paragraphs = split_into_paragraphs(documents, max_sentences, overlap_sentences, embedding_model.tokenizer, max_tokens, overlap_tokens)

print(f"Splitted {len(documents)} documents into {len(paragraphs)} paragraphs with max sequence length {max_tokens}.")
# paragraphs

Splitted 56 documents into 159 paragraphs with max sequence length 512.


Create embeddings for fact paragraphs.

Prefix the paragraphs with the title, if title isn't already included into the paragraph.

In [8]:
embedding_paragraphs = [p.text if p.title in p.text else p.title + '\n' + p.text for p in paragraphs]

embeddings = embedding_model.encode(embedding_paragraphs, convert_to_tensor=True)
# L2-normalize -> dot-score is then same like cosine-similarity
embeddings = embeddings / torch.sqrt((embeddings**2).sum(1, keepdims=True))

print(f"Embedded {len(embeddings)} paragraphs with {embeddings.shape[1]} dimensions each.")

Embedded 159 paragraphs with 1024 dimensions each.


Save paragraphs and embeddings as JSON lines files.

In [9]:
import os

# Export Paragraph data struct as JSON lines file
def write_paragraphs(file: str, paragraphs: list[Paragraph]):
  os.makedirs(os.path.dirname(file), exist_ok=True)
  with open(file, 'w', encoding='utf-8') as f:
    for entry in paragraphs:
      f.write(json.dumps(list(entry.__dict__.values()), ensure_ascii=False) + '\n')

# Export Paragraph Embeddings data struct as JSON lines file
def write_paragraphs_embeddings(file: str, paragraphs_embeddings: torch.Tensor):
  os.makedirs(os.path.dirname(file), exist_ok=True)
  with open(file, 'w', encoding='utf-8') as f:
    for entry in paragraphs_embeddings:
      json.dump(entry.tolist(), f)
      f.write('\n')


write_paragraphs('onprem/paragraphs.json', paragraphs)
write_paragraphs_embeddings('onprem/paragraphs_embeddings.json', embeddings)

Create embedding for question.

In [10]:
question = 'Wer ist antragsberechtigt?'

query_embedding = embedding_model.encode(question, convert_to_tensor=True)
# L2-normalize -> dot-score is then same like cosine-similarity
query_embedding = query_embedding / torch.sqrt((query_embedding**2).sum())

print(f"Embedded 1 question with {len(query_embedding)} dimensions.")

Embedded 1 question with 1024 dimensions.


Some tests for embedding quality:

In [11]:
embedding_paragraphs_test = ['Wer ist antragsberechtigt?', 'Wer ist antragsberechtigt? Test', 'Der Drache ist antragsberechtigt.',]

embeddings_test = embedding_model.encode(embedding_paragraphs_test, convert_to_tensor=True)
# L2-normalize -> dot-score is then same like cosine-similarity
embeddings_test = embeddings_test / torch.sqrt((embeddings_test**2).sum(1, keepdims=True))

scores = embeddings_test @ query_embedding
scores

tensor([1.0000, 0.9519, 0.7401], device='cuda:0')

Find best embeddings via [k-nearest-neighbors (kNN)](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) with [Cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity).

In [12]:
def top_k(query_embedding: torch.Tensor, k: int | None = None) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Calculate the top K similar elements given a query embedding.

    The query_embedding is first L2-normalized. The dot-score is then the same as the cosine-similarity.
    The function returns the similarities and the indices of the top K similar elements.

    Args:
        query_embedding (torch.Tensor): The query embedding.
        k (int, optional): Number of top similar elements to return. If None, all elements are returned. Defaults to None.

    Returns:
        tuple[torch.Tensor, torch.Tensor]: A tuple containing the indices and similarity scores of the top K similar elements.
    """
    scores = embeddings @ query_embedding
    indices = torch.argsort(-scores)
    scores = scores[indices]
    # If k is not None, select only the top k indices and scores
    if k is not None:
        indices = indices[:k]
        scores = scores[:k]
    return indices, scores


k = 25 * 2  # times two: Cross-Encoder Reranking  will reduce again
indices, scores = top_k(query_embedding, k)

print(f"Top {k} paragraphs:")
for i, s in zip(indices, scores):
    print(f"(Score: {s:.4f})  {paragraphs[i]}")

Top 50 paragraphs:
(Score: 0.7058)  Paragraph(id=24, sentence=0, tokens=456, external_id='4.5', title='Welche Angaben sind für die Antragstellung erforderlich?', text='Im Direktantrag und im Antrag über prüfende Dritte sind insbesondere folgende Angaben zu machen, um die Identität und Antragsberechtigung der oder des Antragstellenden sowie die Bemessungsgrundlage festzustellen:\n- Angabe, ob Antrag in eigenem Namen als natürliche Person (Freiberuflerin beziehungsweise Freiberufler oder Gewerbetreibende beziehungsweise Gewerbetreibender) oder durch eine Kapitalgesellschaft/Genossenschaft gestellt wird\n- Name, Geburtsdatum, die beim zuständigen Finanzamt hinterlegte Anschrift, gegebenenfalls Firma und Betriebsstätte\n- Gegebenenfalls steuerliche Identifikationsnummer, Steuernummer und gegebenenfalls Umsatzsteuer-ID\n- Zuständige Finanzämter\n- IBAN der Kontoverbindung, die beim zuständigen Finanzamt für die angegebene steuerliche Identifikationsnummer oder Steuernummer hinterlegt ist\n-

## Re-Ranking and Similarity
Cross-Encoder models calclulate a similarity between question and paragraphs. This enables re-ranking of the found results via embeddings. Embeddings are fast and enable vector indices, but compressing a question or paragraph into a single vector loses a lot of information.

Load Cross-Encoder model.

In [13]:
from sentence_transformers import CrossEncoder

model_x = CrossEncoder('cross-encoder/mmarco-mMiniLMv2-L12-H384-v1')

model_x, model_x.model.config.max_position_embeddings

(<sentence_transformers.cross_encoder.CrossEncoder.CrossEncoder at 0x7fd6f3fa8fa0>,
 514)

In [14]:
def rerank(question: str, paragraphs: list[str],
            min_score: float = -5.0) -> tuple[torch.Tensor, torch.Tensor]:
    scores: torch.Tensor = model_x.predict(
        [[question, p] for p in paragraphs], convert_to_tensor=True)  # type:ignore
    mask = scores >= min_score
    indices = torch.arange(len(scores)).to(scores.device)[mask]
    scores = scores[mask]
    sorted_indices = torch.argsort(-scores)
    return indices[sorted_indices], scores[sorted_indices]


indices_x, scores_x = rerank(question, (embedding_paragraphs[i] for i in indices))

top_paragraphs = [(float(score), paragraphs[indices[index]]) for index, score in zip(indices_x, scores_x)]
# top_paragraphs

Recluster found paragraphs with same title.

In [15]:
def longest_overlap_suffix_prefix(s1: str, s2: str) -> str:
    for i in range(len(s1) - 1, -1, -1):
        if s2.startswith(s1[i:]):
            return s1[i:]
    return ''

def merge_overlapping_fragments(fragments: list[str]) -> str:
    result = fragments[0]
    for i in range(1, len(fragments)):
        overlap = longest_overlap_suffix_prefix(result, fragments[i])
        if len(overlap) == 0:
            result += ' ' + fragments[i]
        else:
            result += fragments[i][len(overlap):]
    return result

def merge_top_paragraphs(top_paragraphs: list[tuple[float, Paragraph]]) -> list[tuple[float, Paragraph]]:
    clustered_paragraphs = {}

    for score, paragraph in top_paragraphs:
        if paragraph.id not in clustered_paragraphs:
            clustered_paragraphs[paragraph.id] = []
        clustered_paragraphs[paragraph.id].append((score, paragraph))

    # Sort each cluster by second entry
    for id in clustered_paragraphs:
        clustered_paragraphs[id] = sorted(clustered_paragraphs[id], key=lambda x: x[1].sentence)

    # Combine clusters in original order
    merged_paragraphs = []
    for id in clustered_paragraphs:
        merged_paragraphs.append(
            (max(score for score, _ in clustered_paragraphs[id]),  # max score over all top paragraph parts
              Paragraph(
                id,
                clustered_paragraphs[id][0][1].sentence,  # take sentence only once from first paragraph part
                0,
                clustered_paragraphs[id][0][1].external_id,  # take external_id only once from first paragraph part
                clustered_paragraphs[id][0][1].title,  # take title only once from first paragraph part
                merge_overlapping_fragments([p[1].text for p in clustered_paragraphs[id]]))))
    return merged_paragraphs


# top_paragraphs = [(float(score), paragraphs[index]) for index, score in zip(indices, scores)]
top_paragraphs = merge_top_paragraphs(top_paragraphs)

top_paragraphs

[(3.7864043712615967,
  Paragraph(id=2, sentence=0, tokens=0, external_id='2.1', title='Wer ist antragsberechtigt?', text='Ein bereits gestellter oder noch zu stellender Antrag auf die Neustarthilfe 2022 für das erste Quartal ist keine Voraussetzung für die Beantragung der Neustarthilfe 2022 für das zweite Quartal. Daher ist es auch möglich, die Neustarthilfe 2022 nur für das zweite Quartal zu beantragen, siehe Ziffer 4.1. Für die Neustarthilfe 2022 grundsätzlich antragsberechtigt sind selbständig erwerbstätige Soloselbständige, Kapitalgesellschaften und Genossenschaften (im Folgenden zusammen mit den Soloselbständigen: Antragstellende) aller Branchen, wenn sie\n- als Soloselbständige ihre selbständige Tätigkeit im Haupterwerb ausüben, das heißt dass der überwiegende Teil der Summe ihrer Einkünfte (mindestens 51 Prozent) aus einer gewerblichen (§ 15 Einkommenssteuergesetz, EStG) und/oder freiberuflichen (§ 18 EStG) Tätigkeit stammt (vergleiche auch 2.4), oder\n    als Ein-Personen-Kapi

Aggregate facts into a text corpus:
*   Use references that can be parsed out of generated response [[x]]
*   Restrict text corpus to max token size (function argument)

In [56]:
import tiktoken

def top_facts(top_paragraphs: list[tuple[float, Paragraph]], tokenizer, max_tokens: int) -> tuple[list[str], int]:
    facts = []
    facts_tokens = 0
    for _, paragraph in top_paragraphs:
        fact = f"[[{paragraph.external_id}]] {paragraph.title} {paragraph.text}"
        fact_tokens = tokenizer.encode(fact)
        if facts_tokens + len(fact_tokens) > max_tokens:
            facts.append(tokenizer.decode(fact_tokens[0:max_tokens - facts_tokens]))
            facts_tokens = max_tokens
            break
        facts.append(fact)
        facts_tokens += 1 + len(fact_tokens)
    return facts, facts_tokens


completion_model = 'gpt-3.5-turbo'
facts, fact_tokens = top_facts(top_paragraphs, tiktoken.encoding_for_model(completion_model), 3000)

print(f"Tokens: {fact_tokens}")
facts

Tokens: 3000


['[[2.1]] Wer ist antragsberechtigt? Ein bereits gestellter oder noch zu stellender Antrag auf die Neustarthilfe 2022 für das erste Quartal ist keine Voraussetzung für die Beantragung der Neustarthilfe 2022 für das zweite Quartal. Daher ist es auch möglich, die Neustarthilfe 2022 nur für das zweite Quartal zu beantragen, siehe Ziffer 4.1. Für die Neustarthilfe 2022 grundsätzlich antragsberechtigt sind selbständig erwerbstätige Soloselbständige, Kapitalgesellschaften und Genossenschaften (im Folgenden zusammen mit den Soloselbständigen: Antragstellende) aller Branchen, wenn sie\n- als Soloselbständige ihre selbständige Tätigkeit im Haupterwerb ausüben, das heißt dass der überwiegende Teil der Summe ihrer Einkünfte (mindestens 51 Prozent) aus einer gewerblichen (§ 15 Einkommenssteuergesetz, EStG) und/oder freiberuflichen (§ 18 EStG) Tätigkeit stammt (vergleiche auch 2.4), oder\n    als Ein-Personen-Kapitalgesellschaft den überwiegenden Teil der Summe der Einkünfte (mindestens 51 Prozent)

# OpenAI Models
Import Open API key.

In [57]:
from google.colab import drive
import openai

drive.mount('/content/drive')
with open('/content/drive/My Drive/Private/api_keys.json', 'r') as f:
    api_keys = json.load(f)

if False:
    # Use models via OpenAI datacenter
    openai.api_key = api_keys['openai']
    engine = None
else:
    # Use models via via Azure (EU)
    openai.api_key = api_keys['azure']
    openai.api_type = 'azure'
    openai.api_base = 'https://techstab-openai.openai.azure.com/'
    openai.api_version = '2023-03-15-preview'
    engine = completion_model.replace('.', '') # Azure needs param engine (no dots)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Open AI Response Generation

In [58]:
def replace(prompt: str, question: str, facts: list[str]) -> str:
    return prompt.replace('{question}', question).replace('{facts}', '\n\n'.join(facts))

def augment_prompt(question: str, facts: list[str]) -> tuple[str, str]:
    system_prompt = """\
Du agierst als KI-Assistent im Antragsportal 'Überbrückungshilfen'.
Du beantwortest dort Fragen, basierend auf einer gegebenen FAQ, zum Teilprogramm 'Neustarthilfe 2022', einem Finanzhilfsprogramm für Soloselbstständige, die im Zeitraum von Januar bis Juni 2021 coronabedingt hohe Umsatzeinbußen hatten.

KONTEXT:
Es folgt ein Auszug aus der Antragsportal-FAQ, der relevante Fakten zur Beantwortung der Frage enthalten könnte.
Die Fakten sind in FAQ-Einträge geclustert, die jeweils mit der Referenzangabe [[x.y]] beginnen.

< BEGIN FAQ >
{facts}
< / END FAQ >

REGELN:
1) Nutze nur die im Kontext gegebenen Fakten für die Antwort, füge kein anderes Wissen hinzu, insbesondere keine Zahlen oder Datumsangaben.
2) Zitiere nach jedem verwendeten Fakt direkt die Referenzangabe [[x.y]] des zugehörigen FAQ-Eintrages.
3) Ignoriere Fragen und irrelevante Fakten im Kontext.
4) Antwortet nur auf Fragen im Themenbereich 'Neustarthilfe'.
5) Wenn kein passender Fakt vorhanden ist, antworte mit: 'Ich bin mir nicht sicher.' und verweise auf ähnliche Fragen, wenn möglich.
6) Beschränke die Antwort auf die relevantesten Aussagen (maximal 3 Sätze) in verständlicher und einfacher Sprache.
7) Verifiziere deine Antwort sorgfältig."""

    prompt = """\
FRAGE:
{question}

ANTWORT:"""

    prompt = f"Frage: {question}\n---\nAntwort:"

    return replace(system_prompt, question, facts), replace(prompt, question, facts)


system_prompt, prompt = augment_prompt(question, facts)

print(system_prompt)
print('###')
print(prompt)
print('###')
print(f"Full Tokens: {len(tiktoken.encoding_for_model(completion_model).encode(system_prompt + prompt))}")

Du agierst als KI-Assistent im Antragsportal 'Überbrückungshilfen'.
Du beantwortest dort Fragen, basierend auf einer gegebenen FAQ, zum Teilprogramm 'Neustarthilfe 2022', einem Finanzhilfsprogramm für Soloselbstständige, die im Zeitraum von Januar bis Juni 2021 coronabedingt hohe Umsatzeinbußen hatten.

KONTEXT:
Es folgt ein Auszug aus der Antragsportal-FAQ, der relevante Fakten zur Beantwortung der Frage enthalten könnte.
Die Fakten sind in FAQ-Einträge geclustert, die jeweils mit der Referenzangabe [[x.y]] beginnen.

< BEGIN FAQ >
[[2.1]] Wer ist antragsberechtigt? Ein bereits gestellter oder noch zu stellender Antrag auf die Neustarthilfe 2022 für das erste Quartal ist keine Voraussetzung für die Beantragung der Neustarthilfe 2022 für das zweite Quartal. Daher ist es auch möglich, die Neustarthilfe 2022 nur für das zweite Quartal zu beantragen, siehe Ziffer 4.1. Für die Neustarthilfe 2022 grundsätzlich antragsberechtigt sind selbständig erwerbstätige Soloselbständige, Kapitalgesells

Call generative language model.
*   'gpt-3.5-turbo' via Chat-API is better now for QA than 'text-davinci-003' via Completion-API
*   'gpt-4' via Chat-API is even better, but slow and expensive

In [59]:
response = openai.ChatCompletion.create(
  model = completion_model if not engine else None,
  engine = engine if engine else None,
  messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': prompt}
    ],
  max_tokens = 500,
  temperature = 0,
  top_p = 0
)

print(response['choices'][0]['message']['content'].strip())
# response

Antragsberechtigt für die Neustarthilfe 2022 sind selbständig erwerbstätige Soloselbständige, Kapitalgesellschaften und Genossenschaften aller Branchen, wenn sie bestimmte Kriterien erfüllen, wie z.B. dass der überwiegende Teil ihrer Einkünfte aus einer gewerblichen und/oder freiberuflichen Tätigkeit stammt und sie keine Fixkostenerstattung in der Überbrückungshilfe IV beantragt oder erhalten haben. [[2.1]]


## Open AI Embeddings
Split into paragraphs, that match the max token sequence length.

In [21]:
max_sentences = 12
overlap_sentences = 2
max_tokens = 8192
overlap_tokens = max_tokens / 6

paragraphs_openai = split_into_paragraphs(documents, max_sentences, overlap_sentences, tiktoken.encoding_for_model('text-embedding-ada-002'), max_tokens, overlap_tokens)

print(f"Splitted {len(documents)} documents into {len(paragraphs_openai)} paragraphs with max sequence length {max_tokens}.")

Splitted 56 documents into 95 paragraphs with max sequence length 8192.


Calculate embeddings. They are already normalized.

In [22]:
embedding_paragraphs_openai = [p.text if p.title in p.text else p.title + ': ' + p.text for p in paragraphs_openai]

if engine:
  # Azure, no batching
  response = [openai.Embedding.create(
    engine = 'text-embedding-ada-002',
    input=e) for e in embedding_paragraphs_openai]
  embeddings_openai = torch.Tensor([obj.data[0]['embedding'] for obj in response])
else:
  # OpenAI
  responses = openai.Embedding.create(
    model = 'text-embedding-ada-002',
    input=embedding_paragraphs_openai)
  embeddings_openai = torch.Tensor([e['embedding'] for e in responses['data']])

Save paragraphs and embeddings as JSON lines files.

In [23]:
write_paragraphs('openai/paragraphs.json', paragraphs_openai)
write_paragraphs_embeddings('openai/paragraphs_embeddings.json', embeddings_openai)

# Aleph Alpha Models
Import Aleph Alpha API key.

In [50]:
import aleph_alpha_client as aleph
from google.colab import drive

drive.mount('/content/drive')
with open('/content/drive/My Drive/Private/api_keys.json', 'r') as f:
    api_keys = json.load(f)

client = aleph.Client(token=api_keys['aleph'])
aleph_completion_model = 'luminous-extended-control'
aleph_embedding_model = 'luminous-base'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Aleph Alpha Response Generation

In [60]:
class AlephTokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def encode(self, text):
        return self.tokenizer.encode(text).ids

    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids)

facts, fact_tokens = top_facts(top_paragraphs, AlephTokenizerWrapper(client.tokenizer(aleph_completion_model)), 1200)  # Aleph Alpha has just an 2048 token window!
system_prompt, prompt = augment_prompt(question, facts)
control_prompt = f"### Instruction:\n{system_prompt}\n\n### Input:\n{question}\n\n### Response:\n"

print(control_prompt)
print(f"Fact Tokens: {fact_tokens}")
print(f"Full Tokens: {len(client.tokenizer(aleph_completion_model).encode(control_prompt))}")

### Instruction:
Du agierst als KI-Assistent im Antragsportal 'Überbrückungshilfen'.
Du beantwortest dort Fragen, basierend auf einer gegebenen FAQ, zum Teilprogramm 'Neustarthilfe 2022', einem Finanzhilfsprogramm für Soloselbstständige, die im Zeitraum von Januar bis Juni 2021 coronabedingt hohe Umsatzeinbußen hatten.

KONTEXT:
Es folgt ein Auszug aus der Antragsportal-FAQ, der relevante Fakten zur Beantwortung der Frage enthalten könnte.
Die Fakten sind in FAQ-Einträge geclustert, die jeweils mit der Referenzangabe [[x.y]] beginnen.

< BEGIN FAQ >
[[2.1]] Wer ist antragsberechtigt? Ein bereits gestellter oder noch zu stellender Antrag auf die Neustarthilfe 2022 für das erste Quartal ist keine Voraussetzung für die Beantragung der Neustarthilfe 2022 für das zweite Quartal. Daher ist es auch möglich, die Neustarthilfe 2022 nur für das zweite Quartal zu beantragen, siehe Ziffer 4.1. Für die Neustarthilfe 2022 grundsätzlich antragsberechtigt sind selbständig erwerbstätige Soloselbständig

In [52]:
response = client.complete(aleph.CompletionRequest(
    aleph.Prompt.from_text(control_prompt),
    maximum_tokens=500), aleph_completion_model)
print(response.completions[0].completion)

[[2.1]] Wer ist antragsberechtigt? Ein bereits gestellter oder noch zu stellender Antrag auf die Neustarthilfe 2022 für das erste Quartal ist keine Voraussetzung für die Beantragung der Neustarthilfe 2022 für das zweite Quartal. Daher ist es auch möglich, die Neustarthilfe 2022 nur für das zweite Quartal zu beantragen, siehe Ziffer 4.1. Für die Neustarthilfe 2022 grundsätzlich antragsberechtigt sind selbständig erwerbstätige Soloselbstständige, Kapitalgesellschaften und Genossenschaften (im Folgenden zusammen mit den Soloselbstständigen: Antragstellende) aller Branchen, wenn sie
- als Soloselbstständige ihre selbständige Tätigkeit im Haupterwerb ausüben, das heißt dass der überwiegende Teil der Summe ihrer Einkünfte (mindestens 51 Prozent) aus einer gewerblichen (§ 15 Einkommenssteuergesetz, EStG) und/oder freiberuflichen (§ 18 EStG) Tätigkeit stammt (vergleiche auch 2.4), oder
 als Ein-Personen-Kapitalgesellschaft den überwiegenden Teil der Summe der Einkünfte (mindestens 51 Prozent) 

## Aleph Alpha Embeddings
See https://docs.aleph-alpha.com/docs/tasks/semantic_embed/ and https://docs.aleph-alpha.com/api/semantic-embed/#semantic-embeddings

Split into paragraphs, that match the max token sequence length.

In [53]:
max_sentences = 12
overlap_sentences = 2
max_tokens = 2048
overlap_tokens = max_tokens / 6

paragraphs_aleph = split_into_paragraphs(documents, max_sentences, overlap_sentences, client.tokenizer(aleph_embedding_model), max_tokens, overlap_tokens)

print(f"Splitted {len(documents)} documents into {len(paragraphs_aleph)} paragraphs with max sequence length {max_tokens}.")

Splitted 56 documents into 95 paragraphs with max sequence length 2048.


Calculate embeddings. They are already normalized with normalize = True. With compress_to_size = 128 the embedding size can be reduced from 5120 (with around 5% accuracy loss).

In [54]:
embedding_paragraphs_aleph = [p.text if p.title in p.text else p.title + ': ' + p.text for p in paragraphs_aleph]

responses = [client.semantic_embed(aleph.SemanticEmbeddingRequest(aleph.Prompt.from_text(p), aleph.SemanticRepresentation.Document, normalize = True), aleph_embedding_model) for p in embedding_paragraphs_aleph]
embeddings_aleph = torch.Tensor([r.embedding for r in responses])

Save paragraphs and embeddings as JSON lines files.

In [55]:
write_paragraphs('aleph/paragraphs.json', paragraphs_aleph)
write_paragraphs_embeddings('aleph/paragraphs_embeddings.json', embeddings_aleph)